In [ ]:
# For Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,recall_score,precision_score

In [ ]:
X = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")
X.head()

In [ ]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['CODE_GENDER'].value_counts()

In [ ]:
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [ ]:
X['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [ ]:
X.head()

In [ ]:

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X:
    if X[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X[col].unique())) <= 2:
            # Train on the training data
            le.fit(X[col])
            # Transform both training and testing data
            X[col] = le.transform(X[col])
           # df_test[col] = le.transform(df_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


In [ ]:
# one-hot encoding of categorical variables
X = pd.get_dummies(X)
#df_test = pd.get_dummies(df_test)
list1=X.columns

print('Training Features shape: ', X.shape)
#print('Testing Features shape: ', df_test.shape)
print(list1)

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(X)
mis_col=list(missing_values.index)
Mis=X[mis_col][:]

for col in mis_col:
    X[col+'_miss']=[0]*len(X[col])
    X[col+'_miss'][list(X[np.isnan(X[col])].index)]=1

print(X.describe)

In [ ]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
print('before...')    
Mis=X[mis_col][:]
print('Mis - ',Mis)

xt = DataFrameImputer().fit_transform(Mis)

print('after...')
print(xt)

In [ ]:
# #Have commented the dropping (not dropping any columns for now) Replace missing with -999

# X_mis=(Mis.isnull().sum() / Mis.shape[0] * 100.00)
# df_mis=pd.DataFrame(np.c_[X_mis.index,X_mis],columns=['Colname','miss_p'])
# drop_col=df_mis[df_mis['miss_p']>65]['Colname'].values
# print('number of columns before dropping->',len(X.columns))
# #X=X.drop(drop_col,axis=1)
# print('number of columns after dropping->',len(X.columns))
# X.replace(np.nan,-999, inplace=True)
# #X.astype('float32',inplace=True)
#print(newX[].describe())
for mis in mis_col:
    X[mis]=xt[mis]
print(X.shape)
print(X.describe)

In [ ]:
# iterate below functions only

In [ ]:
y=X['TARGET']
X=X.drop('TARGET',axis=1)

def split_train_test(X , y):
    return(train_test_split( X , y , test_size=0.2, random_state=42))

X_train, X_test, y_train, y_test =split_train_test(X,y)

In [ ]:
def feature_eng(X_train,y_train,X_test,y_test):
    train_col=list(X_train.columns)
    test_col=list(X_test.columns)    
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X_train, y_train)
    model = SelectFromModel(clf, prefit=True)
    new_trainX = model.transform(X_train)
    n=np.shape(new_trainX)[1]
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    clist=[]
    for f in range(n):     
        clist.append(indices[f])
    new_train_col=[train_col[i] for i in clist]
    new_test_col=[test_col[i] for i in clist]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    df_new_train=pd.DataFrame(X_train.loc[:,new_train_col],columns=new_train_col)            
    df_new_test=pd.DataFrame(X_test.loc[:,new_test_col],columns=new_test_col)
    print("Improved number of features->",len(clist))    
    return(df_new_train,df_new_test)

df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)

In [ ]:
scaler1 = StandardScaler().fit(df_new_train)
df_new_train = scaler1.transform(df_new_train)


scaler2 = StandardScaler().fit(df_new_test)
df_new_test = scaler2.transform(df_new_test)

In [ ]:
def run_model_eval_metric(modelname,modelobj,X_train,Y_train,X_test,Y_test):
    # In order to let the models run faster we use dask dataframe
#     X_train=dd.from_pandas(X_train,npartitions=3)
#     X_test=dd.from_pandas(X_test,npartitions=3)
#     Y_train=dd.from_pandas(Y_train,npartitions=3)
#     Y_test=dd.from_pandas(Y_test,npartitions=3)

    modelobj.fit(X_train,Y_train)
    ypred_train=modelobj.predict(X_train)
    ypred_test=modelobj.predict(X_test)
    y_train_proba = modelobj.predict_proba(X_train)[:,1]
    y_test_proba = modelobj.predict_proba(X_test)[:,1]
    print(modelname,"model:")
    confmat=confusion_matrix(Y_train,ypred_train)
    print("Train Confusion Matrix:")
    print(confmat)
    confmat=confusion_matrix(Y_test,ypred_test)
    print("Test Confusion Matrix:")
    print(confmat)
    metric=pd.DataFrame([[0]*5]*2,columns=['PRECISION','RECALL','F1_SCORE','ROC_AUC_SCORE','ACCURACY'],index=['Train','Test'],dtype='float32')
    metric['PRECISION']['Train']=precision_score(Y_train, ypred_train)
    metric['RECALL']['Train']=recall_score(Y_train, ypred_train)
    metric['F1_SCORE']['Train']=f1_score(Y_train, ypred_train)
    metric['ROC_AUC_SCORE']['Train']=roc_auc_score(Y_train,y_train_proba)
    metric['ACCURACY']['Train']=accuracy_score(Y_train,ypred_train)
    metric['PRECISION']['Test']=precision_score(Y_test, ypred_test)
    metric['RECALL']['Test']=recall_score(Y_test, ypred_test)
    metric['F1_SCORE']['Test']=f1_score(Y_test, ypred_test)
    metric['ROC_AUC_SCORE']['Test']=roc_auc_score(Y_test,y_test_proba)
    metric['ACCURACY']['Test']=accuracy_score(Y_test,ypred_test)
    print(metric)
   
    

In [ ]:
def run_all_models(df_new_train,y_train,df_new_test,y_test):
    # K Nearest Neighbor
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 1)
    run_model_eval_metric('K-Nearest Neighbor 1',knearest,df_new_train,y_train,df_new_test,y_test)
    
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 3)
    run_model_eval_metric('K-Nearest Neighbor 3',knearest,df_new_train,y_train,df_new_test,y_test)

    # Random Forest
    randfor = RandomForestClassifier(n_estimators= 5, max_features= 'auto', max_depth= 70, bootstrap= True, n_jobs= -1)
    run_model_eval_metric('Random Forest',randfor,df_new_train,y_train,df_new_test,y_test)

#     # Decision Trees
#     dectree = DecisionTreeClassifier(criterion="gini",max_depth=15,min_samples_leaf=25)
#     run_model_eval_metric('Decision Tree',dectree,df_new_train,y_train,df_new_test,y_test)

#     # Logistic Regression
#     logreg = LogisticRegression()
#     run_model_eval_metric('Logistic Regression',logreg,df_new_train,y_train,df_new_test,y_test)

#     #SVM
#     svm = SVC()
#     svm_ypred_train,svm_ypred_test=run_model_eval_metric('SVM',svm,df_new_train,y_train,df_new_test,y_test)

run_all_models(df_new_train,y_train,df_new_test,y_test)